In [4]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head()

,Category,Message
0,1,Last summer I had an appointment to get new ti...
1,2,"Friendly staff, same starbucks fair you get an..."
2,1,The food is good. Unfortunately the service is...
3,2,Even when we didn't have a car Filene's Baseme...
4,2,"Picture Billy Joel's \""Piano Man\"" DOUBLED mix..."


# Import Dependencies

In [5]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize message and remove stop words

In [24]:
import nltk
from nltk.tokenize import word_tokenize

df['tokenized_text'] = df['Message'].apply(word_tokenize)

In [25]:
df.head()

,Category,Message,tokenized_text
0,1,Last summer I had an appointment to get new ti...,"[Last, summer, I, had, an, appointment, to, ge..."
1,2,"Friendly staff, same starbucks fair you get an...","[Friendly, staff, ,, same, starbucks, fair, yo..."
2,1,The food is good. Unfortunately the service is...,"[The, food, is, good, ., Unfortunately, the, s..."
3,2,Even when we didn't have a car Filene's Baseme...,"[Even, when, we, did, n't, have, a, car, Filen..."
4,2,"Picture Billy Joel's \""Piano Man\"" DOUBLED mix...","[Picture, Billy, Joel, 's, \, '', Piano, Man\,..."


In [26]:
stopwords=nltk.corpus.stopwords.words('english')

In [32]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [29]:
def remove_stopwords(txt_tokenized):
    tokens_without_sw = [word for word in txt_tokenized if not word in stopwords]
    return tokens_without_sw

df['message_no_sw']=df['tokenized_text'].apply(lambda x:remove_stopwords(x))

In [31]:
df['message_no_sw'][0]

['Last',
 'summer',
 'I',
 'appointment',
 'get',
 'new',
 'tires',
 'wait',
 'super',
 'long',
 'time',
 '.',
 'I',
 'also',
 'went',
 'week',
 'fix',
 'minor',
 'problem',
 'tire',
 'put',
 '.',
 'They',
 '\\',
 "''",
 'fixed\\',
 "''",
 'free',
 ',',
 'next',
 'morning',
 'I',
 'issue',
 '.',
 'I',
 'called',
 'complain',
 ',',
 '\\',
 "''",
 'manager\\',
 "''",
 "n't",
 'even',
 'apologize',
 '!',
 '!',
 '!',
 'So',
 'frustrated',
 '.',
 'Never',
 'going',
 'back',
 '.',
 'They',
 'seem',
 'overpriced',
 ',',
 '.']

# Encode the words and pad the sequences

In [33]:
tokenizer = Tokenizer(num_words=1000, split=" ")
tokenizer.fit_on_texts(df['message_no_sw'].values)

X = tokenizer.texts_to_sequences(df['message_no_sw'].values)
X = pad_sequences(X) # padding our text vector so they all have the same length
X[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [37]:
y = pd.get_dummies(df['Category']).values
[print(df['Category'][i],y[i]) for i in range(0,5)]

1 [1 0]
2 [0 1]
1 [1 0]
2 [0 1]
2 [0 1]


[None, None, None, None, None]

# Create a RNN with LSTM layer

In [38]:
model = Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64, input_length=1000))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 2 units.
model.add(layers.Dense(2))

model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 64)          64000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 163,074
Trainable params: 163,074
Non-trainable params: 0
_________________________________________________________________


# Split the data

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [41]:
batch_size = 32
epochs = 3

history=model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/3
950/950 [==============================] - 283s 298ms/step - loss: nan - accuracy: 0.5000
Epoch 2/3
950/950 [==============================] - 777s 818ms/step - loss: nan - accuracy: 0.5000
Epoch 3/3
950/950 [==============================] - 2157s 2s/step - loss: nan - accuracy: 0.5001


In [42]:
test_loss, test_acc = model.evaluate(X_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

238/238 [==============================] - 0s 981us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Test Loss: 0.0
Test Accuracy: 0.0
